In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import pandas as pd

from insync.db import ListDB
from insync.listregistry import CompletionCommand, ListItem, ListItemProject, ListItemProjectType, ListRegistry


In [3]:
descs = ['eggs', 'milk', 'bread', 'butter']

reg = ListRegistry()

for desc in descs:
    i = ListItem(desc, project=ListItemProject('grocery', ListItemProjectType.CHECKLIST))
    reg.add(i)

milk_item = list(reg.items)[1]
cmd = CompletionCommand(milk_item.uuid, True)

print("\nList:\n",reg, sep='')
print("Cmd before doing:\n\t", cmd)
reg.do(cmd)
print("Cmd after doing:\n\t", cmd)
print("\nList:\n",reg, sep='')

reg.undo()
print("\nList:\n",reg, sep='')


List:
2024-03-01 eggs @^grocery
2024-03-01 milk @^grocery
2024-03-01 bread @^grocery
2024-03-01 butter @^grocery

Cmd before doing:
	 CompletionCommand(uuid=UUID('018dfd1a-eeb3-77bc-ad45-bebdc49d7585'), completed_new=True, completed_orig=None)
Cmd after doing:
	 CompletionCommand(uuid=UUID('018dfd1a-eeb3-77bc-ad45-bebdc49d7585'), completed_new=True, completed_orig=False)

List:
2024-03-01 eggs @^grocery
x 2024-03-01 milk @^grocery
2024-03-01 bread @^grocery
2024-03-01 butter @^grocery


List:
2024-03-01 eggs @^grocery
2024-03-01 milk @^grocery
2024-03-01 bread @^grocery
2024-03-01 butter @^grocery



# Save and restore from the db

In [ ]:
DB_FILE = Path("scratch.db")
DB_FILE.unlink(missing_ok=True)
db = ListDB(DB_FILE)
db.ensure_tables_created()
db.patch(reg)

In [ ]:
reg2 = db.load()

reg.add(ListItem("cheese", project=ListItemProject('grocery', ListItemProjectType.CHECKLIST)))
print("\nOriginal:\n",reg, sep='')
print("\nRestored:\n",reg2, sep='')

In [86]:
import dataclasses


@dataclasses.dataclass(order=True, frozen=True)
class SortKey:
    key: int
    generation: int

@dataclasses.dataclass
class OrderableListItem:
    sort_key: SortKey
    value: int

class OrderableList:
    """test example of a list that can be ordered
    goint to benchmark the how fast the key lenth grows with the number of
    insertions and reorders
    """
    def __init__(self, MIN: int, MAX: int, GAP: int):
        self.items: list[OrderableListItem] = []

        self.MIN: int = MIN
        self.MAX: int = MAX
        self.GAP: int = GAP # span of SortKey

        self.historical_sort_keys: list[SortKey] = []


    @property
    def sorted_items(self) -> list[OrderableListItem]:
        return sorted(self.items, key=lambda x: x.sort_key)

    def get_key(self, after: SortKey, before: SortKey) -> SortKey:
        """Create sort key for a new item that should be inserted between"""

        assert after != before, f"You can never find a gap between a key and itself: {after}."
        assert after.generation == before.generation, f"Can't compare keys from different generations: {after} and {before}"

        if before.key - after.key == 1:
            print(f"No gap between {after} and {before}")
            after, before = self.rebalance(after, before)

        assert after.generation == before.generation, f"Can't compare keys from different generations: {after} and {before}"

        key = after.key + (before.key - after.key) // 2
        return SortKey(key, after.generation)

    def append(self, v: int) -> None:
        """Append an item to the end of the list"""
        after = SortKey(self.MIN, 0) if len(self.items) == 0 else self.items[-1].sort_key
        key = self.get_key(after, SortKey(self.MAX, after.generation))
        oli = OrderableListItem(key, v)
        print(oli)
        assert key not in [i.sort_key for i in self.items], f"No space for {oli} in {self.items}"
        self.items.append(oli)

    # def insert(self, v: int, after: OrderableListItem, before: OrderableListItem) -> None:
    #     """Insert an item between two existing items"""
    #     key = self.get_key(after.sort_key, before.sort_key)
    #     self.items.append(OrderableListItem(key, v))

    def rebalance(self, after: SortKey, before: SortKey) -> tuple[SortKey, SortKey]:
        """Rebalance the sort keys
        Evenly distribute the sort keys starting from 0, with a gap of self.GAP

        Returns the new mapping for after and before
        """

        assert after.generation == before.generation, f"Can't compare keys from different generations: {after} and {before}"
        generation = after.generation + 1

        new_after, new_before = SortKey(after.key, generation), SortKey(before.key, generation)

        for i, oli in enumerate(self.sorted_items):
            new_key = SortKey(i * self.GAP, generation)

            print(f"Rebalancing {oli.sort_key} to {new_key}")
            print(f"after: {after}, before: {before}")

            if oli.sort_key == after:
                new_after = new_key
            if oli.sort_key == before:
                new_before = new_key

            oli.sort_key = new_key

        return new_after, new_before

    ##################################

    def sanity_check_insertion_order(self) -> None:
        """check that the items are in order after intial insertions"""
        from itertools import pairwise
        for a, b in pairwise(self.sorted_items):
            if a.value > b.value:
                raise ValueError(f"Items are out of order: {a} > {b}")

    def sanity_check_key_uniqueness(self) -> None:
        """check that the keys are unique"""
        keys = [i.sort_key for i in self.sorted_items]
        if len(keys) != len(set(keys)):
            raise ValueError("Keys are not unique")



In [87]:

ol = OrderableList(0, 2**8, GAP=5)
for j in range(6):
    ol.append(j)

df = pd.DataFrame(ol.sorted_items)

df

OrderableListItem(sort_key=SortKey(key=128, generation=0), value=0)
OrderableListItem(sort_key=SortKey(key=192, generation=0), value=1)
OrderableListItem(sort_key=SortKey(key=224, generation=0), value=2)
OrderableListItem(sort_key=SortKey(key=240, generation=0), value=3)
OrderableListItem(sort_key=SortKey(key=248, generation=0), value=4)
OrderableListItem(sort_key=SortKey(key=252, generation=0), value=5)


,sort_key,value
0,"{'key': 128, 'generation': 0}",0
1,"{'key': 192, 'generation': 0}",1
2,"{'key': 224, 'generation': 0}",2
3,"{'key': 240, 'generation': 0}",3
4,"{'key': 248, 'generation': 0}",4
5,"{'key': 252, 'generation': 0}",5


In [88]:
ol = OrderableList(0, 2**8, GAP=5)
for j in range(22):
    ol.append(j)

ol.sanity_check_insertion_order()

df = pd.DataFrame(ol.sorted_items)

df

OrderableListItem(sort_key=SortKey(key=128, generation=0), value=0)
OrderableListItem(sort_key=SortKey(key=192, generation=0), value=1)
OrderableListItem(sort_key=SortKey(key=224, generation=0), value=2)
OrderableListItem(sort_key=SortKey(key=240, generation=0), value=3)
OrderableListItem(sort_key=SortKey(key=248, generation=0), value=4)
OrderableListItem(sort_key=SortKey(key=252, generation=0), value=5)
OrderableListItem(sort_key=SortKey(key=254, generation=0), value=6)
OrderableListItem(sort_key=SortKey(key=255, generation=0), value=7)
No gap between SortKey(key=255, generation=0) and SortKey(key=256, generation=0)
Rebalancing SortKey(key=128, generation=0) to SortKey(key=0, generation=1)
after: SortKey(key=255, generation=0), before: SortKey(key=256, generation=0)
Rebalancing SortKey(key=192, generation=0) to SortKey(key=5, generation=1)
after: SortKey(key=255, generation=0), before: SortKey(key=256, generation=0)
Rebalancing SortKey(key=224, generation=0) to SortKey(key=10, generat

,sort_key,value
0,"{'key': 0, 'generation': 2}",0
1,"{'key': 5, 'generation': 2}",1
2,"{'key': 10, 'generation': 2}",2
3,"{'key': 15, 'generation': 2}",3
4,"{'key': 20, 'generation': 2}",4
5,"{'key': 25, 'generation': 2}",5
6,"{'key': 30, 'generation': 2}",6
7,"{'key': 35, 'generation': 2}",7
8,"{'key': 40, 'generation': 2}",8
9,"{'key': 45, 'generation': 2}",9
